# Load the user connection data and connect to the SAP HANA database instance

Before running the next cell make sure [../0x00-setup/temp_user.ini](../0x00-setup/temp_user.ini) is copied to [../0x00-setup/user.ini](../0x00-setup/user.ini) and the user + password are set appropriately.

In [ ]:
from hana_ml.algorithms.pal.utility import Settings
myhost, myport, myuser, mypwd = Settings.load_config("../0x00-setup/user.ini")

In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

# Tables from SAP HANA

In [ ]:
hdf_titanic_train=myconn.table('TRAIN', schema='TITANIC')

# Random Decision Tree classification

aka RDT

🤓 **Let's discuss**:
1. Supervised vs Unsupervised
1. Classification vs Regression
1. Classification algorithms

Random Decision Tree: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.RDTClassifier.html

In [ ]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

UnifiedClassification offers a varity of classfication algorithms. We use RandomDecisionTree for training.

Other options are: 
- 'DecisionTree'
- 'HybridGradientBoostingTree'
- 'LogisticRegression'
- 'MLP'
- 'NaiveBayes'
- 'RandomDecisionTree'
- 'SVM'

### The simplest training call

RDT Classifier has many parameters to influence the execution of the fitting algorithm: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.RDTClassifier.html#rdtclassifier, but for now you run it with the default parameters only.

You will use Unified Classifier class to create an RDT class: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.unified_classification.UnifiedClassification.html#unifiedclassification

In [ ]:
uc_rdt = UnifiedClassification(func='RandomDecisionTree')

The `fit()` procedure returns a fitted object: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.RDTClassifier.html#hana_ml.algorithms.pal.trees.RDTClassifier.fit, ie. populated attributes, like 
- `model_DataFrame`: Trained model content.
- `feature_importances_DataFrame`: The feature importance (the higher, the more important the feature).
- `oob_error_DataFrame`: Out-of-bag error rate or mean squared error for random decision trees up to indexed tree. Set to None if calculate_oob is False.
- `confusion_matrix_DataFrame`: Confusion matrix used to evaluate the performance of classification algorithms.

To understand these structures better check the corresponding PAL documentation: https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/9ad576abab8f497ab2ade2b86c1ff61a.html#_sys_afl.pal_random_decision_trees

To understand the mapping between PAL objects and fields in SQL and in Python, check https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/parameter_mappings.html (or https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/_static/extra_parameter_mappings.html#/ for the full screen)

The simplest training (fit) call: only the key and the label (the target) of the dataset

In [ ]:
uc_rdt.fit(
    data=hdf_titanic_train,
    key='PassengerId', 
    label='Survived'
);

In [ ]:
#Check fit procedure executed on the db side 
print(uc_rdt.get_fit_execute_statement())

You can see SAP HANA's SQLScript wrapper generated for `_SYS_AFL.PAL_UNIFIED_CLASSIFICATION` procedure: https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/8bd88cf29d9a45d0b52daad2653290d4.html#_sys_afl.pal_unified_classification

## Analyze programmatically the Classifier's...

...parameters.

For the complete list of possible parameters: https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/6c931c4a39d24229ae242509a00da9a8.html

In [ ]:
display(uc_rdt.get_parameters())

...output from the fitting procedure

In [ ]:
uc_rdt.model_

...but not all output tables are populated, when there is no split of training data into training and testing dataset (which is the topic of the next exercise)

In [ ]:
for tab in uc_rdt.model_:
    print(f"{tab.select_statement} returns {tab.count()} record(s)")

...model (stored at the position 0)

In [ ]:
hdf_uc_rdt_model=uc_rdt.model_[0]
display(hdf_uc_rdt_model.head(3).select(hdf_uc_rdt_model.columns[2]).collect())

...feature importance (stored at the position 4), or...

In [ ]:
uc_rdt.model_[4].collect()

...or using an attribute `uc_rdt.importance_`

In [ ]:
uc_rdt.importance_.sort('IMPORTANCE', desc=True).collect()

## Analyze visually the Classifier -- by generating a model report

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(uc_rdt).build().display()

🤓 **Let's discuss**:
* The Model Report

## Debrief the model

The raw storage of the model is difficult to interpret...

In [ ]:
uc_rdt.model_[0].head(5).collect()

...but you can use the module `hana_ml.visualizers.model_debriefing` to get a representation of a visualizer for tree model.

Visualize tree model by data in [DOT](https://en.wikipedia.org/wiki/DOT_%28graph_description_language%29) format: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/hana_ml.visualizers.html#hana_ml.visualizers.model_debriefing.TreeModelDebriefing.tree_debrief_with_dot

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
TreeModelDebriefing.tree_debrief_with_dot(uc_rdt.model_[0]);

🤓 **Let's discuss**:
* The Model Debriefing

# Call prediction

In [ ]:
hdf_titanic_test=myconn.table('TEST', schema='TITANIC')

The test table has the same structure except missing the column `Survived`.

In [ ]:
hdf_titanic_test.head(4).collect()

In [ ]:
hdf_res = uc_rdt.predict(hdf_titanic_test, key = 'PassengerId')

In [ ]:
hdf_res.collect()

🤓 **Let's discuss**:
- The structure of the result table `hdf_res`

In [ ]:
display(eval(hdf_res.sort(['CONFIDENCE'], desc=True).head(1).select('REASON_CODE').collect().iloc[0][0]))

In [ ]:
display(eval(hdf_res.sort(['CONFIDENCE'], desc=True).tail(1).select('REASON_CODE').collect().iloc[0][0]))

## Visualize the split of predicted target

In [ ]:
from hana_ml.visualizers.eda import EDAVisualizer

In [ ]:
EDAVisualizer().pie_plot(data=hdf_res, column='SCORE',
                         legend=False, explode=0,
                         startangle=90, counterclock=False
                        );

## Compare to the [ground truth](https://en.wikipedia.org/wiki/Ground_truth#Statistics_and_machine_learning)

In [ ]:
hdf_titanic_complete=myconn.table('COMPLETE', schema='TITANIC')

In [ ]:
hdf_titanic_complete.head(4).collect()

In [ ]:
hdf_titanic_complete.count()

In [ ]:
# Join prediction results with a test table...
hdf_res_ext=hdf_res.set_index('PassengerId').join(hdf_titanic_test.set_index('PassengerId'))

In [ ]:
# ...to get all attributes (variables) and prediction results in one HANA DataFrame
hdf_res_ext.head(3).collect()

In [ ]:
# Join the DataFrame with prediction results and the DataFrame with complete dataset (the ground truth) 
hdf_res_incl_groundtruth=(hdf_res_ext.set_index(['Name', 'Ticket']).join(hdf_titanic_complete.set_index(['name', 'ticket']))
                 .select('PassengerId', 'Name', 'Ticket', 'SCORE','survived',('1-ABS(SCORE-"survived")', 'IS_CORRECT'))
                 .cast('SCORE', 'INT')

)

In [ ]:
hdf_res_incl_groundtruth.head(3).collect()

### Check [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification) of predicted results

In [ ]:
hdf_res_incl_groundtruth.select(('SUM("IS_CORRECT")/COUNT("IS_CORRECT")','Accuracy')).collect()

### Generate the [Confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
hdf_res_incl_groundtruth.agg(agg_list=[('count','PassengerId','COUNT')], group_by=['SCORE','survived']).collect()

In [ ]:
df_res_incl_gt_pivot=(hdf_res_incl_groundtruth
 .pivot_table(values='PassengerId', index='SCORE', columns='survived', aggfunc='count')
 .collect()
);

In [ ]:
# Order columns and rows by 0,1, and remove the Pandas DataFrame index
(df_res_incl_gt_pivot[[df_res_incl_gt_pivot.columns[0]]+sorted(df_res_incl_gt_pivot.columns[1:])]
 .sort_values(by=df_res_incl_gt_pivot.columns[0], axis=0)
 .style.hide(axis='index')
)

## Using `metrics`

In [ ]:
import hana_ml.algorithms.pal.metrics as pal_metrics

Accuracy score: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.metrics.accuracy_score.html#accuracy-score

In [ ]:
pal_metrics.accuracy_score(data=hdf_res_incl_groundtruth, label_true='survived', label_pred='SCORE')

Confusion Matrix: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.metrics.confusion_matrix.html#confusion-matrix

In [ ]:
hdf_cm, hdf_cr = pal_metrics.confusion_matrix(data=hdf_res_incl_groundtruth, key='PassengerId', label_true='survived', label_pred='SCORE')

In [ ]:
hdf_cm.collect()

In [ ]:
hdf_cr.collect()

In [ ]:
from hana_ml.visualizers.metrics import MetricsVisualizer
MetricsVisualizer().plot_confusion_matrix(hdf_cm, normalize=False);

🤓 **Let's discuss**:
- What you have learned so far